In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/intent-recognition-chatbot-corpus-from-askubuntu/AskUbuntu Corpus.json


In [3]:
import json
# read json data as a dictionary 
with open('../input/intent-recognition-chatbot-corpus-from-askubuntu/AskUbuntu Corpus.json', 'r') as f:
  data = json.load(f)
# Intent and Text information are stored in the value corresponding to sentences key 
sentences=data["sentences"]
# Get intent content using list comprehension by looping in the sentences values 
labelList=[i["intent"]for i in sentences]
# Get text content using list comprehension by looping in the sentences values 
textList=[i['text'] for i in sentences]

In [4]:
# Create IntentDataFrame with label list and text list
DFData = {'label' : labelList, 'sentence' : textList}
IntentDataFrame = pd.DataFrame(data = DFData)


In [37]:
OneSentenceDFData = {'label' : 0, 'sentence' : ["hello"]}
OneSentenceDataFrame = pd.DataFrame(data = OneSentenceDFData)

In [38]:
OneSentenceDataFrame

,label,sentence
0,0,hello


In [6]:
IntentDataFrame=IntentDataFrame[IntentDataFrame["label"]!="None"]

In [7]:
IntentDataFrame["label"].value_counts()

Software Recommendation    57
Make Update                47
Shutdown Computer          27
Setup Printer              23
Name: label, dtype: int64

In [8]:
IntentDataFrame.shape

(154, 2)

In [9]:
IntentDataFrame["label"]=IntentDataFrame["label"].map({"Software Recommendation":0,"Make Update":1,"Shutdown Computer":2,"Setup Printer":3})

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
IntentDataFrame["label"]

0      0
1      0
2      0
3      0
4      0
      ..
157    2
158    2
159    2
160    2
161    2
Name: label, Length: 154, dtype: int64

In [11]:
# Split IntentDataFrame into train_df, test_df
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(IntentDataFrame, test_size=0.2,shuffle=True)

In [12]:
#from torch.utils.data import Dataset
!pip install -Uqq datasets
import datasets #Hugging Face library

In [13]:
# convert train_df, test_df to train_dataset, test_dataset
train_dataset=datasets.Dataset.from_pandas(train_df)
test_dataset=datasets.Dataset.from_pandas(test_df)

In [39]:
OneSentenceDataset = datasets.Dataset.from_pandas(OneSentenceDataFrame)

In [31]:
print(train_dataset)

Dataset({
    features: ['label', 'sentence'],
    num_rows: 123
})


In [ ]:
# Only keep columns: label, sentence
train_dataset=train_dataset.remove_columns(["__index_level_0__"])
test_dataset=test_dataset.remove_columns(["__index_level_0__"])

In [15]:
# Import AutoTokenizer with checkpoint"distilbert-base-uncased"
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [16]:
# Tokenization work on train_dataset and test_dataset
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True, padding=True)
tokenize_train=train_dataset.map(preprocess_function,batched=True)
tokenize_test=test_dataset.map(preprocess_function,batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [40]:
Tokenise_OneSentence = OneSentenceDataset.map(preprocess_function,batched=False)

  0%|          | 0/1 [00:00<?, ?ex/s]

In [17]:
# data_collator
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [18]:
# Build model 
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'classifier

In [19]:
# Metrics
from datasets import load_metric
metric = load_metric("accuracy")
def compute_metrics(eval_pred):

    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [20]:
# Model fine tuning training
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenize_train,
    eval_dataset=tokenize_test,
    compute_metrics=compute_metrics,
    

    tokenizer=tokenizer,
    data_collator=data_collator,
)

os.environ["WANDB_DISABLED"] = "true"
trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence.
***** Running training *****
  Num examples = 123
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 80
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.135440,0.612903
2,No log,0.903130,0.645161
3,No log,0.746589,0.645161
4,No log,0.678645,0.741935
5,No log,0.643630,0.774194


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence.
***** Running Evaluation *****
  Num examples = 31
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence.
***** Running Evaluation *****
  Num examples = 31
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence.
***** Running Evaluation *****
  Num examples = 31
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence.
***** Running Evaluation *****
  Num examples = 31
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argu

TrainOutput(global_step=80, training_loss=0.7896259307861329, metrics={'train_runtime': 87.4449, 'train_samples_per_second': 7.033, 'train_steps_per_second': 0.915, 'total_flos': 4137166509840.0, 'train_loss': 0.7896259307861329, 'epoch': 5.0})

In [21]:
# Model evaluation
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence.
***** Running Evaluation *****
  Num examples = 31
  Batch size = 8


{'eval_loss': 0.643629789352417,
 'eval_accuracy': 0.7741935483870968,
 'eval_runtime': 0.872,
 'eval_samples_per_second': 35.551,
 'eval_steps_per_second': 4.587,
 'epoch': 5.0}

In [22]:
tokenize_test

Dataset({
    features: ['label', 'sentence', 'input_ids', 'attention_mask'],
    num_rows: 31
})

In [23]:
# Model save and predict(does not complete)
outputs = trainer.predict(tokenize_test)
y_pred = outputs.predictions.argmax(1)
y_pred

The following columns in the test set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence.
***** Running Prediction *****
  Num examples = 31
  Batch size = 8


array([1, 3, 3, 0, 1, 2, 1, 1, 1, 1, 1, 0, 1, 0, 3, 1, 1, 1, 1, 1, 0, 1,
       0, 3, 1, 0, 0, 0, 0, 0, 0])

In [24]:
# Check if the prediction uses labels and attention_masks
short_tokenize_test = tokenize_test.remove_columns(['label','sentence','attention_mask'])
outputs = trainer.predict(short_tokenize_test)
y_pred = outputs.predictions.argmax(1)
y_pred

***** Running Prediction *****
  Num examples = 31
  Batch size = 8


array([1, 3, 0, 0, 1, 2, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 3, 1, 0, 0, 0, 0, 0, 0])

In [25]:
short_tokenize_test

Dataset({
    features: ['input_ids'],
    num_rows: 31
})

In [26]:
tokenize_test["input_ids"][:][0:1]

[[101,
  2006,
  2428,
  2214,
  1057,
  8569,
  3372,
  2226,
  1020,
  1012,
  5757,
  1011,
  2129,
  2000,
  12200,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]]

In [27]:
#OneTokenisedSentence = tokenize_test["input_ids"][:][0]
#OneSentenceData = {'input_ids' : OneTokenisedSentence}
#OneSentenceDataFrame = pd.DataFrame(data = OneSentenceData)
#OneSentenceDataset=datasets.Dataset.from_pandas(OneSentenceDataFrame)

In [28]:
print(OneSentenceDataset)

Dataset({
    features: ['input_ids'],
    num_rows: 33
})


In [29]:
print(OneSentenceDataset["input_ids"])

[101, 2006, 2428, 2214, 1057, 8569, 3372, 2226, 1020, 1012, 5757, 1011, 2129, 2000, 12200, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [44]:
# get the label prediction for one unique sentence
outputs = trainer.predict(Tokenise_OneSentence)
print(outputs)
y_pred = outputs.predictions.argmax(1)
y_pred

The following columns in the test set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence.
***** Running Prediction *****
  Num examples = 1
  Batch size = 8


PredictionOutput(predictions=array([[ 0.83370376, -0.4297431 ,  0.37688637, -0.5524359 ]],
      dtype=float32), label_ids=array([0]), metrics={'test_loss': 0.7728880643844604, 'test_accuracy': 1.0, 'test_runtime': 0.0412, 'test_samples_per_second': 24.269, 'test_steps_per_second': 24.269})


array([0])

In [45]:
y_pred_series= pd.Series(y_pred) 
y_pred_series=y_pred_series.map({0:"Software Recommendation",1:"Make Update",2:"Shutdown Computer",3:"Setup Printer"})
y_pred_series.values

array(['Software Recommendation'], dtype=object)

In [46]:
y_pred_series[0]

'Software Recommendation'

In [47]:
outputs

PredictionOutput(predictions=array([[ 0.83370376, -0.4297431 ,  0.37688637, -0.5524359 ]],
      dtype=float32), label_ids=array([0]), metrics={'test_loss': 0.7728880643844604, 'test_accuracy': 1.0, 'test_runtime': 0.0412, 'test_samples_per_second': 24.269, 'test_steps_per_second': 24.269})

In [48]:
intent_model=trainer.save_model()

Saving model checkpoint to ./results
Configuration saved in ./results/config.json
Model weights saved in ./results/pytorch_model.bin
tokenizer config file saved in ./results/tokenizer_config.json
Special tokens file saved in ./results/special_tokens_map.json
